# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv('weather_csv.csv', dtype='object', encoding='utf-8')
city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bredasdorp,-34.5322,20.0403,59.29,83,23,5.77,ZA,1636830098
1,saint-philippe,-21.3585,55.7679,76.55,82,100,12.26,RE,1636830098
2,ushuaia,-54.8,-68.3,59.61,36,100,8.59,AR,1636830098
3,tongliao,43.6125,122.2653,30.99,71,14,6.17,CN,1636830099
4,albany,42.6001,-73.9662,56.75,88,92,4.0,US,1636829831
...,...,...,...,...,...,...,...,...,...
541,gaomi,36.3833,119.7528,44.1,66,0,5.26,CN,1636830268
542,pangai,-19.8,-174.35,77.49,81,71,12.64,TO,1636830268
543,kysyl-syr,63.8986,122.7617,-17.37,99,41,4.5,RU,1636830268
544,pinsk,52.1229,26.0951,39.49,76,100,9.64,BY,1636830269


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
#Configuring gmaps
gmaps.configure(api_key=g_key)

#Calling lat/long and humidity
locations = city_df[['Lat', 'Lng']].astype(float)
humidity = city_df['Humidity'].astype(float)

In [13]:
#Creating the map and adding humidity layer
fig = gmaps.figure(zoom_level=2, center=(20,0))

heat_map = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, point_radius=5, opacity=.5)

fig.add_layer(heat_map)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
#Pulling values to filter
temperatures = city_df['Max Temp'].astype(float)
winds = city_df['Wind Speed'].astype(float)
cloudiness = city_df['Cloudiness'].astype(float)
humidity = city_df['Humidity'].astype(float)

#Filtering the original dataset based on my ideal vacation conditions
filtered_df = city_df.loc[(temperatures > 65) & (temperatures < 80)]
filtered_df = filtered_df.loc[(winds < 25)]
filtered_df = filtered_df.loc[(cloudiness < 75)]
filtered_df = filtered_df.loc[(humidity < 80)]
filtered_df.dropna(inplace=True)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
#Creating new DF with hotel column
hotel_df = pd.DataFrame(filtered_df)
hotel_df['Hotel Name']=" "
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
23,safford,32.834,-109.7076,75.94,17,1,3.44,US,1636830105,
26,arraial do cabo,-22.9661,-42.0278,68.72,61,39,13.71,BR,1636830107,
28,castro,-24.7911,-50.0119,71.02,48,0,7.25,BR,1636830107,
29,cape town,-33.9258,18.4232,66.09,70,0,1.01,ZA,1636829823,
31,pacifica,37.6138,-122.4869,74.37,76,20,1.01,US,1636830040,
...,...,...,...,...,...,...,...,...,...,...
501,rio grande,-32.035,-52.0986,75.31,62,68,18.84,BR,1636830256,
509,puerto baquerizo moreno,-0.9,-89.6,67.86,79,38,11.27,EC,1636830012,
511,kieta,-6.2167,155.6333,79.16,79,38,1.43,PG,1636830259,
512,corrales,35.2378,-106.6067,70.34,29,1,3.0,US,1636830259,


In [21]:
#Setting up API call
query_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
parameters = {'radius':5000,
             'keyword':'hotel',
             'key':g_key}

#Iterating through the filtered DF to locate hotels using the Google Places API
for index, row in hotel_df.iterrows():
    lat=row['Lat']
    lng=row['Lng']
    parameters['location'] = f'{lat},{lng}'
    
    response = requests.get(query_url, parameters)
    hotel_list = response.json()
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotel_list['results'][0]['name']
        print(f'Ideal hotel located! Try staying at the {hotel_list["results"][0]["name"]}')
    except (KeyError, IndexError):
        print(f'Oops! No destinations available at {lat}, {lng}')

Ideal hotel located! Try staying at the SpringHill Suites by Marriott Thatcher
Ideal hotel located! Try staying at the Ocean View Hotel
Ideal hotel located! Try staying at the Central Palace Hotel
Ideal hotel located! Try staying at the The Bay Hotel
Ideal hotel located! Try staying at the Pacifica Lighthouse, Trademark Collection by Wyndham
Ideal hotel located! Try staying at the Americas Best Value Inn Sinton
Ideal hotel located! Try staying at the Lobo de Mar
Ideal hotel located! Try staying at the The Redwood Riverwalk, A Boutique Motel
Oops! No destinations available at 14.2, 24.6667
Ideal hotel located! Try staying at the Turis Firper Hotel
Ideal hotel located! Try staying at the Carnarvon Motel
Ideal hotel located! Try staying at the Muscat International Hotel plaza Salalah
Ideal hotel located! Try staying at the Hotel Xaguate
Ideal hotel located! Try staying at the Sofitel Legend Old Cataract Aswan
Ideal hotel located! Try staying at the The Monterey Hotel
Ideal hotel located! 

In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]].astype('float64')

In [25]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))